INTRODUCTION/BUSINESS PROBLEM

In Lagos, Nigeria, Chinese restaurants abound in several locations. However, due to their clientele base over 70% of these restaurants are congregated on the Lagos Island area and some are quite expensive to afford. A new Chinese restaurant franchise wants to make an entry into Lagos(which is the business capital of Nigeria) but would like to understand the terrain before choosing a specific location where they can locate their first branch.

The problems they are faced with right now include;
1. Information of their top 10 competitors presently.
2. Where to locate the restaurant which should be central enough(i.e cinemas, hotels should be close by) and accessiblility to;
  a. young adults in schools,universities.
  b. relatively affordable to all classes in comparison to the Top 10 competitors.

DATASET REQUIRED

To carry out this excercise, I shall be using numerous dataset sourced from Foursquare, Wikipedia amongst others. I will visualise all the current Chinese restaurants located in Lagos presently using the Foursquare API and cluster them into their various neighborhoods.

I will then explore these restaurants and use reviews and tips available using the Foursquare API to determine the Top 10 restaurants.

Afterwards, I will explore the venues around the existing restaurants and determine the best fit for the location of the new Chinese restaurant to be situated.

In [5]:
#Let's import some of the libraries I am going to use

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
import folium
import requests
import random
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
print("Libraries imported")

Libraries imported


In [1]:
#Foursquare API Credentials

CLIENT_ID = 'PUMKFPLVLMDBHJYWINATOBDRDN5FGBEYVJGNRNG2QIFRQDIQ' # my Foursquare ID
CLIENT_SECRET = 'F20KLNJFEZNVQTRNFVAI4Z14IVMT2AMARRESXAT1CHCVVVYW' # my Foursquare Secret
ACCESS_TOKEN = '1XWQ3JFAXHVWDJ3WFUHS0VC0SKXTGEFIZGHXAD5CKPZWAVVZ' # my FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PUMKFPLVLMDBHJYWINATOBDRDN5FGBEYVJGNRNG2QIFRQDIQ
CLIENT_SECRET:F20KLNJFEZNVQTRNFVAI4Z14IVMT2AMARRESXAT1CHCVVVYW


In [4]:
#Let's get the lat and long of Lagos, Nigeria using Geopy
address = 'Lagos,Nigeria'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

6.4550575 3.3941795


In [24]:
search_query = 'Chinese'
radius = 10000
print(search_query + ' .... OK!')

Chinese .... OK!


In [25]:
#Let's define the url that will access the API
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PUMKFPLVLMDBHJYWINATOBDRDN5FGBEYVJGNRNG2QIFRQDIQ&client_secret=F20KLNJFEZNVQTRNFVAI4Z14IVMT2AMARRESXAT1CHCVVVYW&ll=6.4550575,3.3941795&oauth_token=1XWQ3JFAXHVWDJ3WFUHS0VC0SKXTGEFIZGHXAD5CKPZWAVVZ&v=20180604&query=Chinese&radius=10000&limit=30'

In [26]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '607ea47ba08a2a0b752e93c1'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '50ce56fae4b0c28fd3e92607',
    'name': 'Jagzee chow chinese',
    'location': {'address': 'Freedom park',
     'crossStreet': "Opposite St' Nicholas Hospital",
     'lat': 6.449000670882991,
     'lng': 3.3965407536329675,
     'labeledLatLngs': [{'label': 'display',
       'lat': 6.449000670882991,
       'lng': 3.3965407536329675}],
     'distance': 723,
     'cc': 'NG',
     'city': 'Lagos Island',
     'state': 'Lagos',
     'country': 'Nigeria',
     'formattedAddress': ["Freedom park (Opposite St' Nicholas Hospital)",
      'Lagos Island',
      'Lagos']},
    'categories': [{'id': '4bf58dd8d48988d145941735',
      'name': 'Chinese Restaurant',
      'pluralName': 'Chinese Restaurants',
      'shortName': 'Chinese',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
       'suffi

In [15]:
# Let us assign relevant part of JSON to venues
venues = results['response']['venues']

# Let's tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

C:\Users\BEDC\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,venuePage.id,location.postalCode
0,50ce56fae4b0c28fd3e92607,Jagzee chow chinese,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1618912146,False,Freedom park,Opposite St' Nicholas Hospital,6.449001,3.396541,"[{'label': 'display', 'lat': 6.449000670882991...",723,NG,Lagos Island,Lagos,Nigeria,[Freedom park (Opposite St' Nicholas Hospital)...,NaN,NaN,NaN
1,52cbe5b8498e43eff58e7bbd,Chinese Embassy,"[{'id': '4bf58dd8d48988d12c951735', 'name': 'E...",v-1618912146,False,NaN,NaN,6.429962,3.408968,"[{'label': 'display', 'lat': 6.429961852255445...",3237,NG,Lagos,Lagos,Nigeria,"[Lagos, Lagos]",NaN,NaN,NaN
2,52038c20498ed0f5c679d240,Chinese restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1618912146,False,NaN,NaN,6.428109,3.418169,"[{'label': 'display', 'lat': 6.428109370972691...",4005,NG,NaN,NaN,Nigeria,NaN,NaN,NaN,NaN
3,51c1c0ac498ebe224c6d50df,Chinese Cuisine & Fast Food,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1618912146,False,"2, thurborn avenue, off Commercial Rd",NaN,6.507585,3.372843,"[{'label': 'display', 'lat': 6.507585348430234...",6305,NG,Yaba,Lagos,Nigeria,"[2, thurborn avenue, off Commercial Rd, Yaba, ...",NaN,NaN,NaN
4,53540e7411d20c751e55d988,Mr Chong Chinese resturant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1618912146,False,NaN,NaN,6.443529,3.419516,"[{'label': 'display', 'lat': 6.443528856516558...",3082,NG,NaN,NaN,Nigeria,NaN,NaN,NaN,NaN


In [27]:
dataframe.shape

(30, 19)